In [1]:
import numpy as np
import pandas as pd
from lightgbm import LGBMRegressor

pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

df_orig = pd.read_excel("base_table.xlsx")
df_orig.shape

(3499, 43)

In [2]:
df_orig

,Area,AD_Pos_Code,AD_Name,Region,RM_POS,RM_Name,RM_Base Province,RM_Base City,DM_POS,DM_Name,DM_Base Province,DM_Base City,MR_Pos,MR_Name,MR_Base Province,MR_Base City,PT_Group,省份,城市,区县,城市分级,医院编码,医院名称,医院级别,总院关系\n(含引流点),总院关系\n(不含引流点),Hos Segment,医院潜力,医院潜力_Decile,2023Q1 Actual,2023Q2 Actual,2023Q3 Actual,2023Q4 Actual,R6M Sales Actual,MAT Sales Actual,202401 Sales Actual,202402 Sales Actual,2024Q1 Target,2024Q1 Actual,24Q2 Suggested Target,BU 调整,24Q2 Final Target,FTE
0,East,East,East,Dupi.Derm-East1,POS_75775794,姚琳琳,上海,上海市,POS_75800549,周莉清,上海,上海市,POS_75775817,石珍霞,上海,上海市,达必妥皮科专线,上海,上海市,虹口区,T1,AICM,上海市第一人民医院,三级,AICM,AICM,Focus,2735,8,70.240407,156.455233,80.461628,61.500000,130.929181,393.846041,32.768924,22.913454,113.0,113.0,134,39.0,173,0.925134
1,East,East,East,Dupi.Derm-East1,POS_75775794,姚琳琳,上海,上海市,POS_75800549,周莉清,上海,上海市,POS_75775817,石珍霞,上海,上海市,达必妥皮科专线,上海,上海市,虹口区,T1,AICN,上海市第四人民医院,二级,AICM,AICN,Others,1046,4,32.240407,48.000000,59.000000,33.000000,100.000000,197.000000,45.000000,1.000000,63.0,63.0,69,-56.0,13,0.069519
2,East,East,East,Dupi.Derm-East1,POS_75775794,姚琳琳,上海,上海市,POS_75800549,周莉清,上海,上海市,POS_75775817,石珍霞,上海,上海市,达必妥皮科专线,上海,上海市,虹口区,T1,AIGH,上海建工医院,二级,AICM,AIGH,Others,500,3,3.000000,0.000000,4.000000,0.000000,0.000000,7.000000,0.000000,0.000000,21.0,21.0,11,-10.0,1,0.005348
3,East,East,East,Dupi.Derm-East1,POS_75775794,姚琳琳,上海,上海市,POS_75800549,周莉清,上海,上海市,POS_75826028,王文浩,上海,上海市,达必妥皮科专线,上海,上海市,静安区,T1,AIKR,上海市第十人民医院,三级,AIKR,AIKR,Top,1837,7,114.455233,785.822761,-131.647093,150.746802,42.208580,1002.397039,106.149405,2.857140,210.0,210.0,165,20.0,185,0.994624
4,East,East,East,Dupi.Derm-East1,POS_75775794,姚琳琳,上海,上海市,POS_75800549,周莉清,上海,上海市,POS_75775825,吴强勇,上海,上海市,达必妥皮科专线,上海,上海市,静安区,T1,AIKX,上海市闸北区彭浦新村街道社区卫生服务中心,一级,AILZ,AIKX,Others,69,1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,1.0,1,0.0,1,0.001597
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3494,West,West,West,Dupi.Derm-West3,POS_75808496,王江恺,陕西,西安市,POS_75904304,何翡,陕西,西安市,POS_75904359,杨龙,陕西,咸阳市,达必妥混线,陕西,咸阳市,渭城区,T4,CAFX,西藏民族大学附属医院,不详,CAFX,CAFX,Others,304,1,0.000000,0.000000,2.000000,1.000000,2.000000,4.000000,1.000000,0.000000,7.0,7.0,7,0.0,7,0.038251
3495,West,West,West,Dupi.Derm-West3,POS_75808496,王江恺,陕西,西安市,POS_75904304,何翡,陕西,西安市,POS_75809041,郭拴成,陕西,延安市,达必妥皮科专线,陕西,延安市,宝塔区,T4,AXPR,延安大学附属医院,三级,AXPR,AXPR,Focus,3453,9,40.000000,70.000000,86.000000,-19.746802,11.253198,211.253198,26.000000,9.000000,59.0,59.0,99,-20.0,79,0.484663
3496,West,West,West,Dupi.Derm-West3,POS_75808496,王江恺,陕西,西安市,POS_75904304,何翡,陕西,西安市,POS_75809041,郭拴成,陕西,延安市,达必妥皮科专线,陕西,延安市,宝塔区,T4,AXPT,延安市人民医院,三级,AXPT,AXPT,Focus,2504,8,3.000000,0.000000,0.000000,20.000000,27.000000,27.000000,7.000000,0.000000,24.0,24.0,63,-30.0,33,0.202454
3497,West,West,West,Dupi.Derm-West3,POS_75808496,王江恺,陕西,西安市,POS_75904304,何翡,陕西,西安市,POS_75809041,郭拴成,陕西,延安市,达必妥皮科专线,陕西,延安市,宝塔区,T4,AXPV,延安市中医医院,三级,AXPT,AXPV,Others,701,3,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,14.0,14.0,15,0.0,15,0.092025


In [4]:
# 假设 train_full 是你的 DataFrame
columns_list = df_orig.columns.tolist()

print(columns_list)

['Area', 'AD_Pos_Code', 'AD_Name', 'Region', 'RM_POS', 'RM_Name', 'RM_Base Province', 'RM_Base City', 'DM_POS', 'DM_Name', 'DM_Base Province', 'DM_Base City', 'MR_Pos', 'MR_Name', 'MR_Base Province', 'MR_Base City', 'PT_Group', '省份', '城市', '区县', '城市分级', '医院编码', '医院名称', '医院级别', '总院关系\n(含引流点)', '总院关系\n(不含引流点)', 'Hos Segment', '医院潜力', '医院潜力_Decile', '2023Q1 Actual', '2023Q2 Actual', '2023Q3 Actual', '2023Q4 Actual', 'R6M Sales Actual', 'MAT Sales Actual', '202401 Sales Actual', '202402 Sales Actual', '2024Q1 Target', '2024Q1 Actual', ' 24Q2 Suggested Target', 'BU 调整', ' 24Q2 Final Target', 'FTE']


In [3]:
df_orig.rename(columns={' 24Q2 Final Target': '24Q2 Final Target'}, inplace=True)


In [4]:
def check_personnel_deployment(df):
    """
    Check if the personnel deployment adheres to the following principle:
    At the level of provinces/hospital groups/chain groups/other strategic segments defined by the BU, personnel allocation should match the business growth rate. Allocation of personnel with both productivity and growth rates below average should be avoided.
    
    Args:
        df (pandas.DataFrame): The input DataFrame containing the required columns.
        
    Returns:
        pandas.DataFrame: A DataFrame containing the provinces, their total productivity, growth rate, MR count, and a flag indicating violations.
    """
    
    # Calculate total productivity, growth rate, and MR count for each province
    province_stats = df.groupby('省份')[['2023Q2 Actual', '24Q2 Final Target', 'MR_Pos']].agg({'2023Q2 Actual': 'sum', '24Q2 Final Target': 'sum', 'MR_Pos': 'nunique'}).reset_index()
    province_stats['productivity'] = (province_stats['24Q2 Final Target'] / province_stats['MR_Pos']) * 4
    province_stats['growth_rate'] = (province_stats['24Q2 Final Target'] / province_stats['2023Q2 Actual']) - 1
    
    # Calculate the overall average productivity and growth rate using total '24Q2 Final Target' divided by total MR_Pos
    total_24Q2_Final_Target = province_stats['24Q2 Final Target'].sum()
    total_MR_Pos = province_stats['MR_Pos'].sum()
    
    overall_avg_productivity = (total_24Q2_Final_Target / total_MR_Pos) * 4
    overall_avg_growth_rate = (total_24Q2_Final_Target / df['2023Q2 Actual'].sum()) - 1
    
    
    # Check for violations and add a flag column
    province_stats['violation'] = 'N'
    low_productivity_mask = (province_stats['productivity'] < overall_avg_productivity) & (province_stats['growth_rate'] < overall_avg_growth_rate)
    province_stats.loc[low_productivity_mask, 'violation'] = 'Y'
    
    return province_stats

In [5]:
violations_df = check_personnel_deployment(df_orig)

In [6]:
violations_df

,省份,2023Q2 Actual,24Q2 Final Target,MR_Pos,productivity,growth_rate,violation
0,上海,9954.537975,14637,39,1501.230769,0.470385,N
1,云南,2839.521658,4945,15,1318.666667,0.741491,N
2,内蒙古,2481.352907,4318,13,1328.615385,0.740180,N
3,北京,11121.283847,19029,52,1463.769231,0.711043,N
4,吉林,2838.761921,4522,15,1205.866667,0.592948,Y
5,四川,8545.023471,12945,33,1569.090909,0.514917,N
6,天津,2712.160516,4089,14,1168.285714,0.507654,Y
7,宁夏,381.682849,845,3,1126.666667,1.213880,N
8,安徽,1293.625291,3816,12,1272.000000,1.949850,N
9,山东,7441.420353,14547,44,1322.454545,0.954869,N


In [7]:
import pandas as pd

def evaluate_dm_deployment(df):
    """
    Evaluate the deployment of District Managers (DM) based on the following rules:
    1. DM's span of control (number of MRs under their management) should be between 6 and 10.
    2. If the span of control is less than 7, the productivity of the MRs under the DM should not be lower than 70% of the average productivity of all DMs.
    
    Args:
        df (pandas.DataFrame): The input DataFrame containing the required columns.
        
    Returns:
        pandas.DataFrame: A DataFrame containing the DM names, their span of control, span of control range check, overall DM productivity average, actual productivity, and whether they meet the standard or not.
    """
    
    # Calculate the span of control for each DM
    dm_span = df.groupby(['DM_POS', 'DM_Name'])['MR_Pos'].nunique().reset_index()
    dm_span.columns = ['DM_POS', 'DM_Name', 'span_of_control']
    
    # Check if the span of control is within the range of 6-10
    dm_span['span_range_check'] = dm_span['span_of_control'].apply(lambda x: 'Yes' if 6 <= x <= 10 else 'No')
    
    # Calculate the overall productivity for all DMs
    overall_dm_productivity = (df['24Q2 Final Target'].sum() / df['DM_POS'].nunique())
    
    # Calculate the actual productivity for each DM
    dm_productivity = df.groupby('DM_POS')['24Q2 Final Target'].sum().reset_index()
    dm_productivity['productivity'] = dm_productivity['24Q2 Final Target']
    
    # Merge the span of control and productivity
    dm_eval = dm_span.merge(dm_productivity, on='DM_POS', how='left')
    
    # Evaluate the standard
    dm_eval['violation'] = 'No'
    low_span_mask = (dm_eval['span_of_control'] < 7) & (dm_eval['productivity'] < 0.7 * overall_dm_productivity)
    dm_eval.loc[low_span_mask, 'violation'] = 'Yes'
    
    return dm_eval[['DM_POS', 'DM_Name', 'span_of_control', 'span_range_check', 'productivity', 'violation']]

In [8]:
violations_df = evaluate_dm_deployment(df_orig)

In [9]:
violations_df

,DM_POS,DM_Name,span_of_control,span_range_check,productivity,violation
0,POS_75800549,周莉清,7,Yes,2883,No
1,POS_75800550,张帆,7,Yes,1842,No
2,POS_75800551,段梅,7,Yes,2355,No
3,POS_75800552,汪雅湄,7,Yes,3208,No
4,POS_75800553,张俊南,8,Yes,2968,No
...,...,...,...,...,...,...
96,POS_75904304,何翡,6,Yes,1486,Yes
97,POS_75904305,韩子营,5,No,1535,Yes
98,POS_75904306,TBA,7,Yes,1620,No
99,POS_75904307,赵亚星,6,Yes,1818,No


In [30]:
import pandas as pd

def evaluate_rm_deployment(df):
    """
    Evaluate the deployment of Regional Managers (RM) based on the following rules:
    1. RM's span of control (number of DMs under their management) should be between 6 and 10.
    2. If the span of control is less than 7, the sum of productivity of all DMs under the RM should not be lower than 70% of the average productivity of all DMs under all RMs.
    
    Args:
        df (pandas.DataFrame): The input DataFrame containing the required columns.
        
    Returns:
        pandas.DataFrame: A DataFrame containing the RM names, their span of control, span of control range check, overall DM productivity average, actual productivity sum, and whether they meet the standard or not.
    """
    
    # Calculate the span of control for each RM
    rm_span = df.groupby(['RM_POS', 'RM_Name'])['DM_POS'].nunique().reset_index()
    rm_span.columns = ['RM_POS', 'RM_Name', 'span_of_control']
    
    # Check if the span of control is within the range of 6-10
    rm_span['span_range_check'] = rm_span['span_of_control'].apply(lambda x: 'Yes' if 6 <= x <= 8 else 'No')
    
    # Calculate the overall productivity for all DMs under all RMs
    overall_dm_productivity = (df['24Q2 Final Target'].sum() / df['RM_POS'].nunique()) 
    
    # Calculate the actual productivity sum for each RM
    rm_productivity = df.groupby('RM_POS')['24Q2 Final Target'].sum().reset_index()
    rm_productivity['productivity'] = rm_productivity['24Q2 Final Target']
    
    # Merge the span of control and productivity
    rm_eval = rm_span.merge(rm_productivity, on='RM_POS', how='left')
    
    # Evaluate the standard
    rm_eval['violation'] = 'No'
    low_span_mask = (rm_eval['span_of_control'] < 6) & (rm_eval['productivity'] < 0.7 * overall_dm_productivity)
    rm_eval.loc[low_span_mask, 'violation'] = 'Yes'
    
    return rm_eval[['RM_POS', 'RM_Name', 'span_of_control', 'span_range_check', 'productivity', 'violation']]

In [31]:
violations_df = evaluate_rm_deployment(df_orig)

In [32]:
violations_df

,RM_POS,RM_Name,span_of_control,span_range_check,productivity,violation
0,POS_75775794,姚琳琳,6,Yes,14637,No
1,POS_75775795,江军森,5,No,14182,No
2,POS_75775796,孙彦伟,8,Yes,21218,No
3,POS_75775798,丁冬冬,6,Yes,14515,No
4,POS_75775800,张晓丽,8,Yes,18992,No
5,POS_75775801,郭诗文,4,No,7790,Yes
6,POS_75775802,姚锦泉,7,Yes,21557,No
7,POS_75775803,袁晓亮,4,No,8996,Yes
8,POS_75775804,吴姣姣,5,No,13123,No
9,POS_75793103,林颖,6,Yes,12072,No


In [33]:
violations_df[violations_df['violation']=='Yes']

,RM_POS,RM_Name,span_of_control,span_range_check,productivity,violation
5,POS_75775801,郭诗文,4,No,7790,Yes
7,POS_75775803,袁晓亮,4,No,8996,Yes


In [35]:
import pandas as pd

def calculate_pt_group_metrics(df):
    """
    Calculate various metrics for each PT group, including:
    - Number of people in each PT group
    - Total sales for 2023Q1, 2023Q2, 2024Q1, and 24Q2 Final Target
    - Average productivity for 24Q2 and 24Q1
    - Growth rate for 24Q2 and 24Q1
    
    Args:
        df (pandas.DataFrame): The input DataFrame containing the required columns.
        
    Returns:
        pandas.DataFrame: A DataFrame containing the PT group, number of people, total sales, average productivity, and growth rate for different quarters.
    """
    
    # Calculate the number of people in each PT group
    pt_group_count = df.groupby('PT_Group')['MR_Pos'].nunique().reset_index()
    pt_group_count.columns = ['PT_Group', 'num_people']
    
    # Calculate the total sales for different quarters
    pt_group_sales = df.groupby('PT_Group')[['2023Q1 Actual', '2023Q2 Actual', '2024Q1 Actual', '24Q2 Final Target']].sum().reset_index()
    
    # Merge the number of people with the sales data
    pt_group_metrics = pt_group_sales.merge(pt_group_count, on='PT_Group', how='left')
    
    # Calculate the average productivity for 24Q2 and 24Q1
    pt_group_metrics['24Q2_avg_productivity'] = pt_group_metrics['24Q2 Final Target'] / pt_group_metrics['num_people']
    pt_group_metrics['24Q1_avg_productivity'] = pt_group_metrics['2024Q1 Actual'] / pt_group_metrics['num_people']
    
    # Calculate the growth rate for 24Q2 and 24Q1
    pt_group_metrics['24Q2_growth_rate'] = (pt_group_metrics['24Q2 Final Target'] / pt_group_metrics['2023Q2 Actual']) - 1
    pt_group_metrics['24Q1_growth_rate'] = (pt_group_metrics['2024Q1 Actual'] / pt_group_metrics['2023Q1 Actual']) - 1
    
    return pt_group_metrics

In [36]:
pt_grp_eval = calculate_pt_group_metrics(df_orig)

In [37]:
pt_grp_eval

,PT_Group,2023Q1 Actual,2023Q2 Actual,2024Q1 Actual,24Q2 Final Target,num_people,24Q2_avg_productivity,24Q1_avg_productivity,24Q2_growth_rate,24Q1_growth_rate
0,达必妥混线,25712.042601,36772.646611,58749.0,71713,255,281.227451,230.388235,0.950172,1.284883
1,达必妥皮科专线,89773.191404,115657.724663,152344.0,176180,447,394.138702,340.814318,0.523288,0.696988


In [38]:
import pandas as pd

def evaluate_mr_performance(df, pt_group_metrics):
    """
    Evaluate the performance of each Medical Representative (MR) based on the following criteria:
    - Sum up the 2023Q1 Actual, 2023Q2 Actual, 2024Q1 Actual, and 24Q2 Final Target for each MR_Pos
    - Calculate the Q2 Productivity Index and Q2 Growth for each MR_Pos
    - Check if Q2 Productivity Index is lower than 0.5
    - Check if Q2 Growth is lower than the corresponding PT group's 24Q2_growth_rate and if Q2 Productivity Index is between 0.5 and 0.7
    - Calculate similar metrics for Q1

    Args:
        df (pandas.DataFrame): The input DataFrame containing the required columns.
        pt_group_metrics (pandas.DataFrame): The DataFrame containing the metrics for each PT group, calculated by the `calculate_pt_group_metrics` function.

    Returns:
        pandas.DataFrame: A DataFrame containing the MR name, MR position, PT group, summed actual sales, productivity index, growth, and evaluation results for Q2 and Q1.
    """

    # Sum up the actual sales for each MR_Pos
    mr_data = df.groupby(['MR_Pos', 'MR_Name'])[['2023Q1 Actual', '2023Q2 Actual', '2024Q1 Actual', '24Q2 Final Target']].sum().reset_index()

    # Merge the PT_Group column
    mr_data = mr_data.merge(df[['MR_Pos', 'PT_Group']].drop_duplicates(), on='MR_Pos', how='left')

    # Calculate Q2 Productivity Index and Q2 Growth for each MR_Pos
    mr_data = mr_data.merge(pt_group_metrics[['PT_Group', '24Q2_avg_productivity', '24Q2_growth_rate']], on='PT_Group', how='left')
    mr_data['Q2_Productivity_Index'] = mr_data['24Q2 Final Target'] / mr_data['24Q2_avg_productivity']
    mr_data['Q2_Growth'] = (mr_data['24Q2 Final Target'] / mr_data['2023Q2 Actual']) - 1

    # Evaluate Q2 performance
    mr_data['Q2_Productivity_Index_Low'] = (mr_data['Q2_Productivity_Index'] < 0.5).apply(lambda x: 'Yes' if x else 'No')
    mr_data['Q2_Growth_Low_and_Productivity_Index_Medium'] = ((mr_data['Q2_Growth'] < mr_data['24Q2_growth_rate']) & (mr_data['Q2_Productivity_Index'].between(0.5, 0.7))).apply(lambda x: 'Yes' if x else 'No')

    # Calculate Q1 Productivity Index and Q1 Growth for each MR_Pos
    mr_data = mr_data.merge(pt_group_metrics[['PT_Group', '24Q1_avg_productivity', '24Q1_growth_rate']], on='PT_Group', how='left')
    mr_data['Q1_Productivity_Index'] = mr_data['2024Q1 Actual'] / mr_data['24Q1_avg_productivity']
    mr_data['Q1_Growth'] = (mr_data['2024Q1 Actual'] / mr_data['2023Q1 Actual']) - 1

    # Evaluate Q1 performance
    mr_data['Q1_Productivity_Index_Low'] = (mr_data['Q1_Productivity_Index'] < 0.5).apply(lambda x: 'Yes' if x else 'No')
    mr_data['Q1_Growth_Low_and_Productivity_Index_Medium'] = ((mr_data['Q1_Growth'] < mr_data['24Q1_growth_rate']) & (mr_data['Q1_Productivity_Index'].between(0.5, 0.7))).apply(lambda x: 'Yes' if x else 'No')

    return mr_data

In [39]:
violations_df = evaluate_mr_performance(df_orig,pt_grp_eval)

In [50]:
violations_df

,MR_Pos,MR_Name,2023Q1 Actual,2023Q2 Actual,2024Q1 Actual,24Q2 Final Target,PT_Group,24Q2_avg_productivity,24Q2_growth_rate,Q2_Productivity_Index,Q2_Growth,Q2_Productivity_Index_Low,Q2_Growth_Low_and_Productivity_Index_Medium,24Q1_avg_productivity,24Q1_growth_rate,Q1_Productivity_Index,Q1_Growth,Q1_Productivity_Index_Low,Q1_Growth_Low_and_Productivity_Index_Medium
0,POS817474,TBA,172.803053,2.275115e+02,0.0,267,达必妥皮科专线,394.138702,0.523288,0.677426,1.735671e-01,No,Yes,340.814318,0.696988,0.000000,-1.000000,Yes,No
1,POS817475,TBA,100.000000,1.625000e+02,0.0,239,达必妥皮科专线,394.138702,0.523288,0.606386,4.707692e-01,No,Yes,340.814318,0.696988,0.000000,-1.000000,Yes,No
2,POS817476,TBA,42.000000,-1.776357e-15,83.0,152,达必妥混线,281.227451,0.950172,0.540488,-8.556839e+16,No,Yes,230.388235,1.284883,0.360261,0.976190,Yes,No
3,POS817477,TBA,49.949546,6.706818e+01,99.0,191,达必妥皮科专线,394.138702,0.523288,0.484601,1.847848e+00,Yes,No,340.814318,0.696988,0.290481,0.982000,Yes,No
4,POS817478,TBA,62.631686,7.897696e+01,0.0,200,达必妥皮科专线,394.138702,0.523288,0.507436,1.532384e+00,No,No,340.814318,0.696988,0.000000,-1.000000,Yes,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
697,POS_75904426,潘科羽,52.000000,1.020000e+02,160.0,175,达必妥混线,281.227451,0.950172,0.622272,7.156863e-01,No,Yes,230.388235,1.284883,0.694480,2.076923,No,No
698,POS_75905716,管兵,25.240407,6.135422e+01,102.0,181,达必妥混线,281.227451,0.950172,0.643607,1.950083e+00,No,No,230.388235,1.284883,0.442731,3.041139,Yes,No
699,POS_75905717,王嘉霖,88.519186,1.124936e+02,239.0,226,达必妥混线,281.227451,0.950172,0.803620,1.009003e+00,No,No,230.388235,1.284883,1.037379,1.699980,No,No
700,POS_75905718,顾宏雨,38.500000,1.039616e+02,103.0,128,达必妥皮科专线,394.138702,0.523288,0.324759,2.312235e-01,Yes,No,340.814318,0.696988,0.302217,1.675325,Yes,No


In [40]:
violations_df[violations_df['MR_Pos']=='POS_75904360']

,MR_Pos,MR_Name,2023Q1 Actual,2023Q2 Actual,2024Q1 Actual,24Q2 Final Target,PT_Group,24Q2_avg_productivity,24Q2_growth_rate,Q2_Productivity_Index,Q2_Growth,Q2_Productivity_Index_Low,Q2_Growth_Low_and_Productivity_Index_Medium,24Q1_avg_productivity,24Q1_growth_rate,Q1_Productivity_Index,Q1_Growth,Q1_Productivity_Index_Low,Q1_Growth_Low_and_Productivity_Index_Medium
666,POS_75904360,TBA,101.0,154.0,146.0,174,达必妥混线,281.227451,0.950172,0.618716,0.12987,No,Yes,230.388235,1.284883,0.633713,0.445545,No,Yes


In [10]:
import pandas as pd

def evaluate_mr_city_coverage(df):
    """
    Evaluate the city coverage and base city alignment for each Medical Representative (MR) based on the following criteria:
    - Calculate the sum of 24Q2 Final Target and hospital potential for each MR in different cities
    - Count the number of cities covered by each MR and check if it's greater than 3
    - Find the city with the highest 24Q2 Final Target and the city with the highest hospital potential for each MR
    - Check if the MR_Base City matches the city with the highest 24Q2 Final Target or the city with the highest hospital potential

    Args:
        df (pandas.DataFrame): The input DataFrame containing the required columns.

    Returns:
        pandas.DataFrame: A DataFrame containing the MR information, city, city coverage, top cities, and base city alignment evaluation.
    """

    # Extract the required columns for each MR
    mr_data = df[['MR_Pos', 'MR_Name', 'MR_Base City', '省份', '城市', 'R6M Sales Actual', '医院潜力']].copy()

    # Calculate the sum of 24Q2 Final Target and hospital potential for each MR in different cities
    mr_city_summary = mr_data.groupby(['MR_Pos', 'MR_Name', '省份', '城市', 'MR_Base City'])[['R6M Sales Actual', '医院潜力']].sum().reset_index()

    # Count the number of cities covered by each MR
    mr_city_count = mr_city_summary.groupby('MR_Pos')['城市'].nunique().reset_index()
    mr_city_count.columns = ['MR_Pos', 'num_cities_covered']
    mr_city_count['multi_city_coverage'] = mr_city_count['num_cities_covered'].apply(lambda x: 'Yes' if x > 3 else 'No')

    # Find the city with the highest 24Q2 Final Target and the city with the highest hospital potential for each MR
    mr_top_sales_city = mr_city_summary.loc[mr_city_summary.groupby('MR_Pos')['R6M Sales Actual'].idxmax()][['MR_Pos', '城市']]
    mr_top_sales_city.columns = ['MR_Pos', 'top_sales_city']

    mr_top_potential_city = mr_city_summary.loc[mr_city_summary.groupby('MR_Pos')['医院潜力'].idxmax()][['MR_Pos', '城市']]
    mr_top_potential_city.columns = ['MR_Pos', 'top_potential_city']

    # Merge the city coverage, top cities, and base city information
    mr_evaluation = mr_city_summary.merge(mr_city_count, on='MR_Pos', how='left')
    mr_evaluation = mr_evaluation.merge(mr_top_sales_city, on='MR_Pos', how='left')
    mr_evaluation = mr_evaluation.merge(mr_top_potential_city, on='MR_Pos', how='left')

    # Evaluate the base city alignment
    mr_evaluation['base_city_aligned'] = 'No'
    mr_evaluation.loc[mr_evaluation['MR_Base City'] == mr_evaluation['top_sales_city'], 'base_city_aligned'] = 'Yes'
    mr_evaluation.loc[mr_evaluation['MR_Base City'] == mr_evaluation['top_potential_city'], 'base_city_aligned'] = 'Yes'

    return mr_evaluation[['MR_Pos', 'MR_Name', 'MR_Base City','省份', '城市', 'R6M Sales Actual', '医院潜力', 'num_cities_covered', 'multi_city_coverage', 'top_sales_city', 'top_potential_city', 'base_city_aligned']]



In [11]:
violations_df = evaluate_mr_city_coverage(df_orig)

In [12]:
violations_df

,MR_Pos,MR_Name,MR_Base City,省份,城市,R6M Sales Actual,医院潜力,num_cities_covered,multi_city_coverage,top_sales_city,top_potential_city,base_city_aligned
0,POS817474,TBA,重庆市,重庆,重庆市,236.005088,18925,1,No,重庆市,重庆市,Yes
1,POS817475,TBA,重庆市,重庆,重庆市,360.283419,12028,1,No,重庆市,重庆市,Yes
2,POS817476,TBA,重庆市,重庆,重庆市,141.746802,5332,1,No,重庆市,重庆市,Yes
3,POS817477,TBA,重庆市,重庆,重庆市,137.114811,17902,1,No,重庆市,重庆市,Yes
4,POS817478,TBA,昆明市,云南,昆明市,368.526746,679,1,No,昆明市,昆明市,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...
825,POS_75904426,潘科羽,黔西南布依族苗族自治州,贵州,黔西南布依族苗族自治州,15.493605,4108,2,No,六盘水市,六盘水市,No
826,POS_75905716,管兵,淮安市,江苏,淮安市,106.208430,14653,1,No,淮安市,淮安市,Yes
827,POS_75905717,王嘉霖,苏州市,江苏,苏州市,287.803053,7508,1,No,苏州市,苏州市,Yes
828,POS_75905718,顾宏雨,衢州市,浙江,衢州市,233.992297,3145,1,No,衢州市,衢州市,Yes


In [13]:
violations_df[violations_df['MR_Pos']=='POS_75869524']

,MR_Pos,MR_Name,MR_Base City,省份,城市,R6M Sales Actual,医院潜力,num_cities_covered,multi_city_coverage,top_sales_city,top_potential_city,base_city_aligned
706,POS_75869524,吴永琴,玉溪市,云南,楚雄彝族自治州,253.531977,5297,2,No,楚雄彝族自治州,楚雄彝族自治州,No
707,POS_75869524,吴永琴,玉溪市,云南,玉溪市,51.000000,3477,2,No,楚雄彝族自治州,楚雄彝族自治州,No


In [19]:
violations_df[violations_df['base_city_aligned']=='No']

,MR_Pos,MR_Name,MR_Base City,省份,城市,24Q2 Final Target,医院潜力,num_cities_covered,multi_city_coverage,top_sales_city,top_potential_city,base_city_aligned
223,POS_75809049,汪国飞,合肥市,安徽,合肥市,91,5281,2,No,滁州市,滁州市,No
224,POS_75809049,汪国飞,合肥市,安徽,滁州市,112,8671,2,No,滁州市,滁州市,No
263,POS_75809083,邱智灿,南平市,福建,三明市,184,8687,2,No,三明市,三明市,No
264,POS_75809083,邱智灿,南平市,福建,南平市,181,5828,2,No,三明市,三明市,No
526,POS_75830465,赵阳,酒泉市,甘肃,张掖市,26,2445,4,Yes,武威市,武威市,No
527,POS_75830465,赵阳,酒泉市,甘肃,武威市,87,2834,4,Yes,武威市,武威市,No
528,POS_75830465,赵阳,酒泉市,甘肃,酒泉市,14,1632,4,Yes,武威市,武威市,No
529,POS_75830465,赵阳,酒泉市,甘肃,金昌市,85,1952,4,Yes,武威市,武威市,No
625,POS_75869378,卢月莉,汕尾市,广东,揭阳市,125,4141,2,No,揭阳市,揭阳市,No
626,POS_75869378,卢月莉,汕尾市,广东,汕尾市,77,2073,2,No,揭阳市,揭阳市,No


In [18]:
mr_city_summary = df_orig.groupby(['MR_Pos', 'MR_Name', '省份', '城市'])[['R6M Sales Actual', '医院潜力']].sum().reset_index()

In [19]:
mr_city_summary[mr_city_summary['MR_Pos']=='POS_75904426']

,MR_Pos,MR_Name,省份,城市,R6M Sales Actual,医院潜力
824,POS_75904426,潘科羽,贵州,六盘水市,117.480814,7080
825,POS_75904426,潘科羽,贵州,黔西南布依族苗族自治州,15.493605,4108


In [78]:
def evaluate_dm_city_coverage(df):
    """
    Evaluate the city coverage and base city alignment for each District Manager (DM).
    
    Args:
        df (pandas.DataFrame): The input DataFrame containing the required columns.
    
    Returns:
        pandas.DataFrame: A DataFrame containing the DM information, city coverage, top cities, 
        base city alignment evaluation, and cross-province coverage.
    """
    # Extract the required columns for each DM
    dm_data = df[['DM_POS', 'DM_Name', 'DM_Base City', 'DM_Base Province', '省份', '城市', 'R6M Sales Actual', '医院潜力', '医院编码']].copy()
    
    # Calculate the sum of R6M Sales Actual for each DM in different cities (without deduplication)
    sales_summary = dm_data.groupby(['DM_POS', 'DM_Name', 'DM_Base Province', '省份', '城市', 'DM_Base City'])['R6M Sales Actual'].sum().reset_index()
    
    # Calculate the sum of unique hospital potential for each DM in different cities
    potential_summary = dm_data.drop_duplicates(subset=['DM_POS', '城市', '医院编码']).groupby(['DM_POS', 'DM_Name', 'DM_Base Province', '省份', '城市', 'DM_Base City'])['医院潜力'].sum().reset_index()
    
    # Merge sales and potential summaries
    dm_city_summary = sales_summary.merge(potential_summary, on=['DM_POS', 'DM_Name', 'DM_Base Province', '省份', '城市', 'DM_Base City'], how='outer')
    
    # Count the number of cities covered by each DM
    dm_city_count = dm_city_summary.groupby('DM_POS')['城市'].nunique().reset_index()
    dm_city_count.columns = ['DM_POS', 'num_cities_covered']
    
    # Find the city with the highest R6M Sales Actual and the city with the highest hospital potential for each DM
    dm_top_sales_city = dm_city_summary.loc[dm_city_summary.groupby('DM_POS')['R6M Sales Actual'].idxmax()][['DM_POS', '城市']]
    dm_top_sales_city.columns = ['DM_POS', 'top_sales_city']
    
    dm_top_potential_city = dm_city_summary.loc[dm_city_summary.groupby('DM_POS')['医院潜力'].idxmax()][['DM_POS', '城市']]
    dm_top_potential_city.columns = ['DM_POS', 'top_potential_city']
    
    # Merge the city coverage, top cities, and base city information
    dm_evaluation = dm_city_summary.merge(dm_city_count, on='DM_POS', how='left')
    dm_evaluation = dm_evaluation.merge(dm_top_sales_city, on='DM_POS', how='left')
    dm_evaluation = dm_evaluation.merge(dm_top_potential_city, on='DM_POS', how='left')
    
    # Evaluate the base city alignment
    dm_evaluation['base_city_aligned'] = 'No'
    dm_evaluation.loc[dm_evaluation['DM_Base City'] == dm_evaluation['top_sales_city'], 'base_city_aligned'] = 'Yes'
    dm_evaluation.loc[dm_evaluation['DM_Base City'] == dm_evaluation['top_potential_city'], 'base_city_aligned'] = 'Yes'
    
    # Check if the DM covers cities outside their base province
    dm_evaluation['cross_province'] = dm_evaluation.apply(lambda row: 'Yes' if row['DM_Base Province'] != row['省份'] else 'No', axis=1)
    
    # Calculate cross_province_all
    cross_province_all = dm_evaluation.groupby('DM_POS')['cross_province'].apply(lambda x: 'Yes' if 'Yes' in x.values else 'No').reset_index()
    cross_province_all.columns = ['DM_POS', 'cross_province_all']
    
    # Merge cross_province_all into dm_evaluation
    dm_evaluation = dm_evaluation.merge(cross_province_all, on='DM_POS', how='left')
    
    return dm_evaluation[['DM_POS', 'DM_Name', 'DM_Base City', 'DM_Base Province', '省份', '城市', 'R6M Sales Actual', '医院潜力', 
                          'num_cities_covered', 'top_sales_city', 'top_potential_city', 'base_city_aligned', 'cross_province', 'cross_province_all']]

In [79]:
violations_df = evaluate_dm_city_coverage(df_orig)

In [80]:
violations_df

,DM_POS,DM_Name,DM_Base City,DM_Base Province,省份,城市,R6M Sales Actual,医院潜力,num_cities_covered,top_sales_city,top_potential_city,base_city_aligned,cross_province,cross_province_all
0,POS_75800549,周莉清,上海市,上海,上海,上海市,4033.719045,34055,1,上海市,上海市,Yes,No,No
1,POS_75800550,张帆,南京市,江苏,江苏,南京市,2335.519403,36105,1,南京市,南京市,Yes,No,No
2,POS_75800551,段梅,苏州市,江苏,江苏,苏州市,3239.647935,42518,1,苏州市,苏州市,Yes,No,No
3,POS_75800552,汪雅湄,杭州市,浙江,浙江,杭州市,4415.917969,49073,1,杭州市,杭州市,Yes,No,No
4,POS_75800553,张俊南,温州市,浙江,浙江,温州市,3309.769691,50863,1,温州市,温州市,Yes,No,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,POS_75904307,赵亚星,成都市,四川,四川,成都市,989.238174,15234,5,成都市,成都市,Yes,No,No
376,POS_75904307,赵亚星,成都市,四川,四川,攀枝花市,615.429651,4665,5,成都市,成都市,Yes,No,No
377,POS_75904307,赵亚星,成都市,四川,四川,资阳市,20.240407,9290,5,成都市,成都市,Yes,No,No
378,POS_75904307,赵亚星,成都市,四川,四川,遂宁市,271.250582,6516,5,成都市,成都市,Yes,No,No


In [82]:
violations_df[violations_df['DM_POS']=='POS_75800549']

,DM_POS,DM_Name,DM_Base City,DM_Base Province,省份,城市,R6M Sales Actual,医院潜力,num_cities_covered,top_sales_city,top_potential_city,base_city_aligned,cross_province,cross_province_all
0,POS_75800549,周莉清,上海市,上海,上海,上海市,4033.719045,34055,1,上海市,上海市,Yes,No,No


In [89]:
violations_df[violations_df['DM_POS']=='POS_75849945']

,DM_POS,DM_Name,DM_Base City,DM_Base Province,省份,城市,R6M Sales Actual,医院潜力,num_cities_covered,top_sales_city,top_potential_city,base_city_aligned,cross_province,cross_province_all
262,POS_75849945,闫玮,西安市,陕西,陕西,宝鸡市,1188.872036,10227,6,宝鸡市,西安市,Yes,No,Yes
263,POS_75849945,闫玮,西安市,陕西,陕西,渭南市,198.227616,4976,6,宝鸡市,西安市,Yes,No,Yes
264,POS_75849945,闫玮,西安市,陕西,陕西,西安市,997.858056,17557,6,宝鸡市,西安市,Yes,No,Yes
265,POS_75849945,闫玮,西安市,陕西,陕西,铜川市,48.000000,1746,6,宝鸡市,西安市,Yes,No,Yes
266,POS_75849945,闫玮,西安市,陕西,青海,海东市,0.000000,598,6,宝鸡市,西安市,Yes,Yes,Yes
267,POS_75849945,闫玮,西安市,陕西,青海,西宁市,441.238682,14446,6,宝鸡市,西安市,Yes,Yes,Yes


In [84]:
violations_df[violations_df['base_city_aligned']=='No']

,DM_POS,DM_Name,DM_Base City,DM_Base Province,省份,城市,R6M Sales Actual,医院潜力,num_cities_covered,top_sales_city,top_potential_city,base_city_aligned,cross_province,cross_province_all
24,POS_75800559,王少卿,广州市,广东,广东,云浮市,2.000000,1634,4,佛山市,佛山市,No,No,No
25,POS_75800559,王少卿,广州市,广东,广东,佛山市,1073.959441,43152,4,佛山市,佛山市,No,No,No
26,POS_75800559,王少卿,广州市,广东,广东,江门市,222.000000,10527,4,佛山市,佛山市,No,No,No
27,POS_75800559,王少卿,广州市,广东,广东,肇庆市,134.421144,9743,4,佛山市,佛山市,No,No,No
103,POS_75810174,李炳翰,武汉市,湖北,湖北,仙桃市,64.493605,2857,12,荆州市,荆州市,No,No,No
104,POS_75810174,李炳翰,武汉市,湖北,湖北,咸宁市,159.000000,3854,12,荆州市,荆州市,No,No,No
105,POS_75810174,李炳翰,武汉市,湖北,湖北,孝感市,584.942442,5796,12,荆州市,荆州市,No,No,No
106,POS_75810174,李炳翰,武汉市,湖北,湖北,宜昌市,696.791316,12257,12,荆州市,荆州市,No,No,No
107,POS_75810174,李炳翰,武汉市,湖北,湖北,恩施土家族苗族自治州,394.365698,8346,12,荆州市,荆州市,No,No,No
108,POS_75810174,李炳翰,武汉市,湖北,湖北,潜江市,78.493605,2532,12,荆州市,荆州市,No,No,No


In [14]:
def evaluate_rm_coverage(df):
    """
    Evaluate the coverage and alignment for each Regional Manager (RM) at the province level.
    
    Args:
        df (pandas.DataFrame): The input DataFrame containing the required columns.
    
    Returns:
        pandas.DataFrame: A DataFrame containing the RM information, coverage, top provinces, 
        base province and city alignment evaluation, and province sharing.
    """
    # Extract the required columns for each RM
    rm_data = df[['RM_POS', 'RM_Name', 'RM_Base City', 'RM_Base Province', '省份', 'R6M Sales Actual', '医院潜力', '医院编码']].copy()
    
    # Calculate the sum of R6M Sales Actual for each RM in different provinces (without deduplication)
    sales_summary = rm_data.groupby(['RM_POS', 'RM_Name', 'RM_Base City', 'RM_Base Province', '省份'])['R6M Sales Actual'].sum().reset_index()
    
    # Calculate the sum of unique hospital potential for each RM in different provinces
    potential_summary = rm_data.drop_duplicates(subset=['RM_POS', '省份', '医院编码']).groupby(['RM_POS', 'RM_Name', 'RM_Base City', 'RM_Base Province', '省份'])['医院潜力'].sum().reset_index()
    
    # Merge sales and potential summaries
    rm_province_summary = sales_summary.merge(potential_summary, on=['RM_POS', 'RM_Name', 'RM_Base City', 'RM_Base Province', '省份'], how='outer')
    
    # Count the number of provinces covered by each RM
    rm_province_count = rm_province_summary.groupby('RM_POS')['省份'].nunique().reset_index()
    rm_province_count.columns = ['RM_POS', 'num_provinces_covered']
    
    # Find the province with the highest R6M Sales Actual and the province with the highest hospital potential for each RM
    rm_top_sales_province = rm_province_summary.loc[rm_province_summary.groupby('RM_POS')['R6M Sales Actual'].idxmax()][['RM_POS', '省份']]
    rm_top_sales_province.columns = ['RM_POS', 'top_sales_province']
    
    rm_top_potential_province = rm_province_summary.loc[rm_province_summary.groupby('RM_POS')['医院潜力'].idxmax()][['RM_POS', '省份']]
    rm_top_potential_province.columns = ['RM_POS', 'top_potential_province']
    
    # Merge the province coverage, top provinces, and base province information
    rm_evaluation = rm_province_summary.merge(rm_province_count, on='RM_POS', how='left')
    rm_evaluation = rm_evaluation.merge(rm_top_sales_province, on='RM_POS', how='left')
    rm_evaluation = rm_evaluation.merge(rm_top_potential_province, on='RM_POS', how='left')
    
    # Evaluate the base province alignment
    rm_evaluation['base_province_aligned'] = 'No'
    rm_evaluation.loc[rm_evaluation['RM_Base Province'] == rm_evaluation['top_sales_province'], 'base_province_aligned'] = 'Yes'
    rm_evaluation.loc[rm_evaluation['RM_Base Province'] == rm_evaluation['top_potential_province'], 'base_province_aligned'] = 'Yes'
    
    # Check if the RM covers provinces outside their base province
    rm_evaluation['cross_province'] = rm_evaluation.apply(lambda row: 'Yes' if row['RM_Base Province'] != row['省份'] else 'No', axis=1)
    
    # Calculate cross_province_all
    cross_province_all = rm_evaluation.groupby('RM_POS')['cross_province'].apply(lambda x: 'Yes' if 'Yes' in x.values else 'No').reset_index()
    cross_province_all.columns = ['RM_POS', 'cross_province_all']
    
    # Merge cross_province_all into rm_evaluation
    rm_evaluation = rm_evaluation.merge(cross_province_all, on='RM_POS', how='left')
    
    # Check if a province has more than 2 different RM_POS
    province_rm_count = rm_evaluation.groupby('省份')['RM_POS'].nunique().reset_index()
    province_rm_count['multiple_rms'] = province_rm_count['RM_POS'].apply(lambda x: 'Yes' if x >= 2 else 'No')
    
    # Merge the multiple_rms information into rm_evaluation
    rm_evaluation = rm_evaluation.merge(province_rm_count[['省份', 'multiple_rms']], on='省份', how='left')
    
    return rm_evaluation[['RM_POS', 'RM_Name', 'RM_Base City', 'RM_Base Province', '省份', 'R6M Sales Actual', '医院潜力', 
                          'num_provinces_covered', 'top_sales_province', 'top_potential_province', 'base_province_aligned', 'cross_province', 'cross_province_all', 'multiple_rms']]

In [104]:
violations_df = evaluate_rm_coverage(df_orig)
violations_df

,RM_POS,RM_Name,RM_Base City,RM_Base Province,省份,R6M Sales Actual,医院潜力,num_provinces_covered,top_sales_province,top_potential_province,base_province_aligned,cross_province,cross_province_all,multiple_rms
0,POS_75775794,姚琳琳,上海市,上海,上海,19772.169357,224342,1,上海,上海,Yes,No,No,No
1,POS_75775795,江军森,杭州市,浙江,浙江,17911.347279,273664,1,浙江,浙江,Yes,No,No,Yes
2,POS_75775796,孙彦伟,南京市,江苏,江苏,29250.457935,386583,1,江苏,江苏,Yes,No,No,No
3,POS_75775798,丁冬冬,广州市,广东,广东,16657.149697,292242,2,广东,广东,Yes,No,Yes,Yes
4,POS_75775798,丁冬冬,广州市,广东,海南,1628.502963,36966,2,广东,广东,Yes,Yes,Yes,No
5,POS_75775800,张晓丽,沈阳市,辽宁,内蒙古,6591.715463,84849,3,辽宁,辽宁,Yes,Yes,Yes,No
6,POS_75775800,张晓丽,沈阳市,辽宁,山西,4714.294307,91013,3,辽宁,辽宁,Yes,Yes,Yes,No
7,POS_75775800,张晓丽,沈阳市,辽宁,辽宁,14259.106653,215420,3,辽宁,辽宁,Yes,No,Yes,No
8,POS_75775801,郭诗文,天津市,天津,天津,4482.454633,95519,2,天津,河北,Yes,No,Yes,No
9,POS_75775801,郭诗文,天津市,天津,河北,3777.010192,181702,2,天津,河北,Yes,Yes,Yes,No


In [105]:
violations_df[violations_df['multiple_rms']=='Yes']

,RM_POS,RM_Name,RM_Base City,RM_Base Province,省份,R6M Sales Actual,医院潜力,num_provinces_covered,top_sales_province,top_potential_province,base_province_aligned,cross_province,cross_province_all,multiple_rms
1,POS_75775795,江军森,杭州市,浙江,浙江,17911.347279,273664,1,浙江,浙江,Yes,No,No,Yes
3,POS_75775798,丁冬冬,广州市,广东,广东,16657.149697,292242,2,广东,广东,Yes,No,Yes,Yes
13,POS_75775803,袁晓亮,北京市,北京,北京,13736.068900,112297,1,北京,北京,Yes,No,No,Yes
16,POS_75793103,林颖,深圳市,广东,广东,9854.130650,174176,2,广东,广东,Yes,No,Yes,Yes
27,POS_75850230,苗广亮,北京市,北京,北京,15476.402122,153914,1,北京,北京,Yes,No,No,Yes
30,POS_75868977,叶斌,宁波市,浙江,浙江,14858.933211,257911,1,浙江,浙江,Yes,No,No,Yes


In [106]:
violations_df[violations_df['RM_POS']=='POS_75775800']

,RM_POS,RM_Name,RM_Base City,RM_Base Province,省份,R6M Sales Actual,医院潜力,num_provinces_covered,top_sales_province,top_potential_province,base_province_aligned,cross_province,cross_province_all,multiple_rms
5,POS_75775800,张晓丽,沈阳市,辽宁,内蒙古,6591.715463,84849,3,辽宁,辽宁,Yes,Yes,Yes,No
6,POS_75775800,张晓丽,沈阳市,辽宁,山西,4714.294307,91013,3,辽宁,辽宁,Yes,Yes,Yes,No
7,POS_75775800,张晓丽,沈阳市,辽宁,辽宁,14259.106653,215420,3,辽宁,辽宁,Yes,No,Yes,No


In [18]:
import numpy as np
import pandas as pd
from lightgbm import LGBMRegressor

pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

df = pd.read_excel("ss0_filter-Copy1.xlsx")
df.shape

(3972, 122)

In [22]:
def evaluate_rm_coverage(df):
    """
    Evaluate the coverage and alignment for each regional Manager (RM) at the province level.
    
    Args:
        df (pandas.DataFrame): The input DataFrame containing the required columns.
    
    Returns:
        pandas.DataFrame: A DataFrame containing the RM information, coverage, top provinces, 
        base province and city alignment evaluation, and province sharing.
    """
    # Extract the required columns for each RM
    rm_data = df[['rm_position_cd', 'rm_name', 'rm_base_city_name', 'rm_base_province_name', 'hco_province_name', 'r6m_actual_sales', 'hco_potential_value', 'hco_cd']].copy()
    
    # Calculate the sum of r6m_actual_sales for each RM in different provinces (without deduplication)
    sales_summary = rm_data.groupby(['rm_position_cd', 'rm_name', 'rm_base_city_name', 'rm_base_province_name', 'hco_province_name'])['r6m_actual_sales'].sum().reset_index()
    
    # Calculate the sum of unique hospital potential for each RM in different provinces
    potential_summary = rm_data.drop_duplicates(subset=['rm_position_cd', 'hco_province_name', 'hco_cd']).groupby(['rm_position_cd', 'rm_name', 'rm_base_city_name', 'rm_base_province_name', 'hco_province_name'])['hco_potential_value'].sum().reset_index()
    
    # Merge sales and potential summaries
    rm_province_summary = sales_summary.merge(potential_summary, on=['rm_position_cd', 'rm_name', 'rm_base_city_name', 'rm_base_province_name', 'hco_province_name'], how='outer')
    
    # Count the number of provinces covered by each RM
    rm_province_count = rm_province_summary.groupby('rm_position_cd')['hco_province_name'].nunique().reset_index()
    rm_province_count.columns = ['rm_position_cd', 'num_provinces_covered']
    
    # Find the province with the highest r6m_actual_sales and the province with the highest hospital potential for each RM
    rm_top_sales_province = rm_province_summary.loc[rm_province_summary.groupby('rm_position_cd')['r6m_actual_sales'].idxmax()][['rm_position_cd', 'hco_province_name']]
    rm_top_sales_province.columns = ['rm_position_cd', 'top_sales_province']
    
    rm_top_potential_province = rm_province_summary.loc[rm_province_summary.groupby('rm_position_cd')['hco_potential_value'].idxmax()][['rm_position_cd', 'hco_province_name']]
    rm_top_potential_province.columns = ['rm_position_cd', 'top_potential_province']
    
    # Merge the province coverage, top provinces, and base province information
    rm_evaluation = rm_province_summary.merge(rm_province_count, on='rm_position_cd', how='left')
    rm_evaluation = rm_evaluation.merge(rm_top_sales_province, on='rm_position_cd', how='left')
    rm_evaluation = rm_evaluation.merge(rm_top_potential_province, on='rm_position_cd', how='left')
    
    # Evaluate the base province alignment
    rm_evaluation['base_province_aligned'] = 'No'
    rm_evaluation.loc[rm_evaluation['rm_base_province_name'] == rm_evaluation['top_sales_province'], 'base_province_aligned'] = 'Yes'
    rm_evaluation.loc[rm_evaluation['rm_base_province_name'] == rm_evaluation['top_potential_province'], 'base_province_aligned'] = 'Yes'
    
    # Check if the RM covers provinces outside their base province
    rm_evaluation['cross_province'] = rm_evaluation.apply(lambda row: 'Yes' if row['rm_base_province_name'] != row['hco_province_name'] else 'No', axis=1)
    
    # Calculate cross_province_all
    cross_province_all = rm_evaluation.groupby('rm_position_cd')['cross_province'].apply(lambda x: 'Yes' if 'Yes' in x.values else 'No').reset_index()
    cross_province_all.columns = ['rm_position_cd', 'cross_province_all']
    
    # Merge cross_province_all into rm_evaluation
    rm_evaluation = rm_evaluation.merge(cross_province_all, on='rm_position_cd', how='left')
    
    # Check if a province has more than 2 different rm_position_cd
    province_rm_count = rm_evaluation.groupby('hco_province_name')['rm_position_cd'].nunique().reset_index()
    province_rm_count['multiple_rms'] = province_rm_count['rm_position_cd'].apply(lambda x: 'Yes' if x >= 2 else 'No')
    
    # Merge the multiple_rms information into rm_evaluation
    rm_evaluation = rm_evaluation.merge(province_rm_count[['hco_province_name', 'multiple_rms']], on='hco_province_name', how='left')
    
    return rm_evaluation[['rm_position_cd', 'rm_name', 'rm_base_city_name', 'rm_base_province_name', 'hco_province_name', 'r6m_actual_sales', 'hco_potential_value', 
                          'num_provinces_covered', 'top_sales_province', 'top_potential_province', 'base_province_aligned', 'cross_province', 'cross_province_all', 'multiple_rms']]
    
violations_df = evaluate_rm_coverage(df_orig)
violations_df

,rm_position_cd,rm_name,rm_base_city_name,rm_base_province_name,hco_province_name,r6m_actual_sales,hco_potential_value,num_provinces_covered,top_sales_province,top_potential_province,base_province_aligned,cross_province,cross_province_all,multiple_rms


In [27]:
rm_data

,rm_position_cd,rm_name,rm_base_city_name,rm_base_province_name,hco_province_name,r6m_actual_sales,hco_potential_value,hco_cd
0,POS_75775794,姚琳琳,NaN,NaN,上海,8.335906e+05,1527.0,AIPF
1,POS_75775794,姚琳琳,NaN,NaN,上海,3.477193e+05,397.0,D81Y
2,POS_75775794,姚琳琳,NaN,NaN,上海,NaN,116.0,D81Y
3,POS_75775794,姚琳琳,NaN,NaN,上海,NaN,115.0,K1X0
4,POS_75775794,姚琳琳,NaN,NaN,上海,2.674764e+04,30.0,K1X0
...,...,...,...,...,...,...,...,...
3967,POS_75775804,吴姣姣,NaN,NaN,四川,NaN,615.0,AUZV
3968,POS_75775804,吴姣姣,NaN,NaN,四川,5.248220e+04,3047.0,AUZW
3969,POS_75838607,饶亮,NaN,NaN,湖北,1.428314e+06,1714.0,AQFU
3970,POS_75838607,饶亮,NaN,NaN,湖北,3.420186e+05,4.0,JCE1


In [11]:
column_mapping = [
    ('areaad_position_cd', 'AreaAD_Pos_Code'),
    ('ad_name', 'AD_Name'),
    ('region', 'Region'),
    ('rm_position_cd', 'RM_POS'),
    ('rm_name', 'RM_Name'),
    ('rm_base_province_name', 'RM_Base Province'),
    ('rm_base_city_name', 'RM_Base City'),
    ('dm_pos', 'DM_POS'),
    ('dm_name', 'DM_Name'),
    ('dm_base_province_name', 'DM_Base Province'),
    ('dm_base_city_name', 'DM_Base City'),
    ('mr_pos', 'MR_Pos'),
    ('mr_name', 'MR_Name'),
    ('mr_base_province_name', 'MR_Base Province'),
    ('mr_base_city_name', 'MR_Base City'),
    ('pt_group', 'PT_Group'),
    ('hco_province_name', '省份'),
    ('hco_city_name', '城市'),
    ('hco_county_name', '区县'),
    ('hco_city_tier', '城市分级'),
    ('hco_cd', '医院编码'),
    ('hco_name', '医院名称'),
    ('hco_tier', '医院级别'),
    ('real_hco_cd', '医院或者药房'),
    ('real_hco_nm', '医院或者药房名称'),
    ('real_hco_type', 'Y列的机构属性'),
    ('hco_segment', 'Hos Segment'),
    ('hco_potential_value', '医院潜力'),
    ('hco_potential_decile', '医院潜力_Decile'),
    ('product_cd', 'Brand'),
    ('lyq1_actual_sales', '2023Q1 Actual'),
    ('lyq2_actual_sales', '2023Q2 Actual'),
    ('lyq3_actual_sales', '2023Q3 Actual'),
    ('lyq4_actual_sales', '2023Q4 Actual'),
    ('r6m_actual_sales', 'R6M Sales Actual'),
    ('mat_actual_sales', 'MAT Sales Actual'),
    ('rq_actual_sales', 'RQ Sales Actual'),
    ('tym1_actual_sales', '202401 Sales Actual'),
    ('tym2_actual_sales', '202402 Sales Actual'),
    ('tym3_actual_sales', '202403 Sales Actual'),
    ('tym4_actual_sales', '202404 Sales Actual'),
    ('tym5_actual_sales', '202405 Sales Actual'),
    ('tym6_actual_sales', '202406 Sales Actual'),
    ('tym7_actual_sales', '202407 Sales Actual'),
    ('tym8_actual_sales', '202408 Sales Actual'),
    ('tym9_actual_sales', '202409 Sales Actual'),
    ('tym10_actual_sales', '2024010 Sales Actual'),
    ('tym11_actual_sales', '2024011 Sales Actual'),
    ('tym12_actual_sales', '2024012 Sales Actual'),
    ('tyq1_target', '2024Q1 Target'),
    ('tyq2_target', '24Q2 Final Target'),
    ('fte', 'FTE'),
    ('tyq1_actual_sales','2024Q1 Actual')
]

In [28]:
column_mapping

[('areaad_position_cd', 'AreaAD_Pos_Code'),
 ('ad_name', 'AD_Name'),
 ('region', 'Region'),
 ('rm_position_cd', 'RM_POS'),
 ('rm_name', 'RM_Name'),
 ('rm_base_province_name', 'RM_Base Province'),
 ('rm_base_city_name', 'RM_Base City'),
 ('dm_pos', 'DM_POS'),
 ('dm_name', 'DM_Name'),
 ('dm_base_province_name', 'DM_Base Province'),
 ('dm_base_city_name', 'DM_Base City'),
 ('mr_pos', 'MR_Pos'),
 ('mr_name', 'MR_Name'),
 ('mr_base_province_name', 'MR_Base Province'),
 ('mr_base_city_name', 'MR_Base City'),
 ('pt_group', 'PT_Group'),
 ('hco_province_name', '省份'),
 ('hco_city_name', '城市'),
 ('hco_county_name', '区县'),
 ('hco_city_tier', '城市分级'),
 ('hco_cd', '医院编码'),
 ('hco_name', '医院名称'),
 ('hco_tier', '医院级别'),
 ('real_hco_cd', '医院或者药房'),
 ('real_hco_nm', '医院或者药房名称'),
 ('real_hco_type', 'Y列的机构属性'),
 ('hco_segment', 'Hos Segment'),
 ('hco_potential_value', '医院潜力'),
 ('hco_potential_decile', '医院潜力_Decile'),
 ('product_cd', 'Brand'),
 ('lyq1_actual_sales', '2023Q1 Actual'),
 ('lyq2_actual_sales'

In [9]:
def replace_column_names(file_path, column_mapping):
    # 读取文件内容
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()

    # 执行替换
    for old_name, new_name in column_mapping:
        content = content.replace(f"'{new_name}'", f"'{old_name}'")
        content = content.replace(f'"{new_name}"', f'"{old_name}"')
        # 如果列名在代码中没有被引号包围，我们也可以直接替换
        content = content.replace(new_name, old_name)

    # 将修改后的内容写回文件
    with open(file_path, 'w', encoding='utf-8') as file:
        file.write(content)

    print(f"File {file_path} has been updated.")

# 使用函数
file_path = 'ffe_groundrules_test.py'
replace_column_names(file_path, column_mapping)

File ffe_groundrules_test.py has been updated.


In [10]:
import pandas as pd

def rename_excel_columns(file_path, column_mapping, sheet_name="Sheet1"):
    # 读取 Excel 文件
    df = pd.read_excel(file_path, sheet_name=sheet_name)
    
    # 创建一个字典，用于 rename 方法
    rename_dict = {new: old for old, new in column_mapping}
    
    # 重命名列
    df = df.rename(columns=rename_dict)
    
    # 将修改后的 DataFrame 保存回 Excel 文件
    df.to_excel(file_path, index=False, sheet_name=sheet_name)
    
    print(f"File {file_path} has been updated.")

# 使用函数
file_path = 'base_table copy.xlsx'
rename_excel_columns(file_path, column_mapping)

File base_table copy.xlsx has been updated.
